In [1]:
import pandas as pd
import numpy as np
import torch
from gensim.models import Word2Vec
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

2024-10-29 14:35:05.599103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 14:35:09.038336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-29 14:35:09.038384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-29 14:35:09.424962: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 14:35:13.784800: W tensorflow/stream_executor/platform/de

In [2]:
#chessData = pd.read_csv('Data/chessData.csv')
#random_evals = pd.read_csv('Data/random_evals.csv')
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [3]:
len(tactic_evals)
tactic_evals = tactic_evals[:500000]


In [4]:
tactic_evals_check = tactic_evals.dropna()
tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))

predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
predictors_raw.head()

/tmp/ipykernel_4792/2706941485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
/tmp/ipykernel_4792/2706941485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))


,FEN,Move
0,r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...,d3g6
1,r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1P/PPQN1PP1/...,c2g6
2,r2qkbr1/pb1nn3/1ppp2Bp/8/3P1p2/2P2N1P/PPQN1PP1...,g8g6
3,8/7p/R5p1/2p1pkP1/7P/P4PK1/1r6/3q4 w - - 6 46,a6f6
4,6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...,h3g4


In [5]:
def separate_board(board):
    return board.replace('/', ' ').split()

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [6]:
tokens_lines = list(map(separate_board, np.asarray(tactic_evals_check['FEN'])))
tokens_lines = [line[0:8] for line in tokens_lines]

tokens_moves = list(map(separate_move, np.asarray(tactic_evals_check['Move'])))
#tokens_piece = list(map(fen_to_tokens_without_spaces, np.asarray(tactic_evals['FEN'])))
predictors_raw['FEN'] = tokens_lines
predictors_raw['Move'] = tokens_moves


In [7]:
fen_embedding_dim = 32
fen_embeddings = Word2Vec(sentences=tokens_lines, vector_size=fen_embedding_dim, window=5, min_count=1, sg=1)
fen_embeddings.train(tokens_lines, total_examples=len(tokens_lines), epochs=10)

(31168852, 37201280)

In [8]:
move_embedding_dim = 32
move_embeddings = Word2Vec(sentences=tokens_moves, vector_size=move_embedding_dim, window=5, min_count=1, sg=1)
move_embeddings.train(tokens_moves, total_examples=len(tokens_moves), epochs=10)

(2647264, 18619900)

In [9]:
fen_data = []
move_data = []

for line in range(len(tokens_lines)):
    fen_data.append(fen_embeddings.wv[tokens_lines[line]])

for move in range(len(tokens_moves)):
    move_data.append(move_embeddings.wv[tokens_moves[move]])

predictors = pd.DataFrame({
    'fen_data': fen_data,
    'move_data': move_data
})

In [13]:
move_data_expanded = []
for embedding in move_data:
    if embedding.shape == (4, 32):
        move_data_expanded.append(np.vstack([embedding, np.zeros((4, 32))]))
    else:
        move_data_expanded.append(np.vstack([embedding, np.zeros((3, 32))]))
        
predictors['move_data'] = move_data_expanded

In [14]:
print(f"fen_data_shape: {predictors['fen_data'][0].shape}\n move_data_shape: {predictors['move_data'][0].shape}")


fen_data_shape: (8, 32)
 move_data_shape: (8, 32)


In [15]:
fen_data = np.array(predictors['fen_data'].tolist())
move_data = np.array(predictors['move_data'].tolist()) 


print(fen_data.shape) 
print(move_data.shape) 


X_fen = np.reshape(fen_data, (fen_data.shape[0], 8, 32, 1)) 
X_move = np.reshape(move_data, (move_data.shape[0], 8, 32, 1)) 

y = tactic_evals_check['isCheck'].to_numpy()  


print(X_fen.shape) 
print(X_move.shape)  
print(y.shape)  

(465016, 8, 32)
(465016, 8, 32)
(465016, 8, 32, 1)
(465016, 8, 32, 1)
(465016,)


In [ ]:
predictors.head()

In [18]:
ncols = predictors.shape[1]

fen_input = Input(shape=(8, 32, 1), name='fen_input')
move_input = Input(shape=(8, 32, 1), name='move_input')

fen_conv1 = Conv2D(16, (2,2), activation='relu')(fen_input)
move_conv1 = Conv2D(16, (2,2), activation='relu')(move_input)

fen_mp1 = MaxPooling2D(pool_size=(2,2))(fen_conv1)
move_mp1 = MaxPooling2D(pool_size=(2,2))(move_conv1)

fen_flatter = Flatten()(fen_mp1)
move_flatter = Flatten()(move_mp1)

combined = Concatenate()([fen_flatter, move_flatter])

dense = (Dense(32, activation = 'relu'))(combined)
output = (Dense(1 , activation = 'sigmoid'))(dense)

mate_evaluator = Model(inputs=[fen_input, move_input], outputs=output)
mate_evaluator.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
		      metrics=['accuracy'])

mate_evaluator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fen_input (InputLayer)         [(None, 8, 32, 1)]   0           []                               
                                                                                                  
 move_input (InputLayer)        [(None, 8, 32, 1)]   0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 7, 31, 16)    80          ['fen_input[0][0]']              
                                                                                                  
 conv2d_3 (Conv2D)              (None, 7, 31, 16)    80          ['move_input[0][0]']             
                                                                                            

In [19]:
#acc = 0.8509
early_stopping_monitor = EarlyStopping(patience=2)
mate_evaluator.fit([X_fen, X_move], y, 
                    validation_split=0.3, 
                    epochs=10, 
                    callbacks=[early_stopping_monitor])

2024-10-29 14:45:55.521439: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 333323264 exceeds 10% of free system memory.


Epoch 1/10
10173/10173 [==============================] - 60s 6ms/step - loss: 0.2816 - accuracy: 0.8840 - val_loss: 0.2768 - val_accuracy: 0.8887
Epoch 2/10
10173/10173 [==============================] - 57s 6ms/step - loss: 0.2540 - accuracy: 0.8974 - val_loss: 0.2603 - val_accuracy: 0.8950
Epoch 3/10
10173/10173 [==============================] - 58s 6ms/step - loss: 0.2428 - accuracy: 0.9026 - val_loss: 0.2505 - val_accuracy: 0.8996
Epoch 4/10
10173/10173 [==============================] - 57s 6ms/step - loss: 0.2356 - accuracy: 0.9057 - val_loss: 0.2500 - val_accuracy: 0.9001
Epoch 5/10
10173/10173 [==============================] - 59s 6ms/step - loss: 0.2316 - accuracy: 0.9076 - val_loss: 0.2444 - val_accuracy: 0.9028
Epoch 6/10
10173/10173 [==============================] - 59s 6ms/step - loss: 0.2280 - accuracy: 0.9086 - val_loss: 0.2463 - val_accuracy: 0.9024
Epoch 7/10
10173/10173 [==============================] - 60s 6ms/step - loss: 0.2246 - accuracy: 0.9106 - val_loss: 0

In [1]:
test_match_board = 'R3K1NR/PPPBBPPP/8/k7/3P2pp/p3P3/1pQ1p3/1r4nr'
test_match_mate = '2da5'

sep_board = separate_board(test_match_board)
sep_mate = separate_move(test_match_mate)


board_emb = fen_embeddings.wv[sep_board]
mate_emb = move_embeddings.wv[sep_mate]

print(board_emb.shape)
print(mate_emb.shape)

#mate_evaluator.predict([])

NameError: name 'separate_board' is not defined